In [1]:
# -------------------------------
#Importing Dependencies
# -------------------------------
import csv
import os
# Organisation Libraries 
import pandas as pd
import numpy as np

# SQL Libraries
from sqlalchemy import create_engine
import psycopg2
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

import time
from getpass import getpass

# ML Libraryies
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
# Using psycopg2 to pull from the NBA_Analysis DB
password = getpass()
cn = psycopg2.connect(host = "127.0.0.1", port = "5432", database = "NBA_Analysis", user = "postgres", password = password)
cur = cn.cursor()
cur.execute("Select * FROM nba_data")
query_results1 = cur.fetchall()

cur.execute("Select * FROM players")
query_results2 = cur.fetchall()


In [ ]:
cur.execute("Select * FROM nba_data LIMIT 0")
colnames1 = [desc[0] for desc in cur.description]
cur.execute("Select * FROM players LIMIT 0")
colnames2 = [desc[0] for desc in cur.description]

In [ ]:
sum_df = pd.DataFrame(query_results1)
sum_df.columns = colnames1
sum_df.head()


player_df = pd.DataFrame(query_results2)
player_df.columns = colnames2
player_df.shape

In [ ]:
inertia = []
k = list(range(1, 11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(sum_df)
    inertia.append(km.inertia_)

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)